# Combineren en manipuleren van gegevens

De verzamelde data moet gecombineerd worden voordat deze gemodelleerd kan worden. De gegevens van SolarEdge, het weer en de stand van de zon komen in één tabel.

- Solaredge: Dates are always in the time zone of the site. De tijdstippen van SolarEdge worden (voor nu) met 1 uur gecompenseerd.
- KNMI: Tijdstippen in UTC

**Uitleg kolommen van KNMI**

Het KNMI levert veel gegevens aan, helaas zitten niet al deze gegevens in de weersvoorspelling. De vetgedrukte items, temperatuur, duur van de neerslag en bewolkingsgraad worden overgenomen. De rest van de kolommen worden hier niet verder gebruikt. 

- DD: Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Meer info

- FH: Uurgemiddelde windsnelheid (in 0.1 m/s). Meer info

- FF: Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur

- FX: Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak

- **T**: Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming

- T10N: Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur

- TD: Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming

- SQ: Duur van de zonneschijn (in 0.1 uren) per uurvak, berekend uit globale straling (-1 for <0.05 uur)

- Q: Globale straling (in J/cm2) per uurvak

- **DR**: Duur van de neerslag (in 0.1 uur) per uurvak

- RH: Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)

- P: Luchtdruk (in 0.1 hPa) herleid naar zeeniveau, tijdens de waarneming

- VV: Horizontaal zicht tijdens de waarneming (0=minder dan 100m, 1=100-200m, 2=200-300m,..., 49=4900-5000m, 50=5-6km, 56=6-7km, 57=7-8km, ..., 79=29-30km, 80=30-35km, 81=35-40km,..., 89=meer dan 70km)

- **N**: Bewolking (bedekkingsgraad van de bovenlucht in achtsten), tijdens de waarneming (9=bovenlucht onzichtbaar)

- U: Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming

- WW: Weercode (00-99), visueel(WW) of automatisch(WaWa) waargenomen, voor het actuele weer of het weer in het afgelopen uur. Meer info

- IX: Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2,3=bemand en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen), 5,6=automatisch en weggelaten (geen belangrijk weersverschijnsel, geen gegevens), 7=automatisch gebruikmakend van code uit automatische waarnemingen)

- M: Mist 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- R: Regen 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- S: Sneeuw 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- O: Onweer 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming

- Y: IJsvorming 0=niet voorgekomen, 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming



In [8]:
import sqlite3
import pysolar
import pandas as pd


# Inlezen gegevens solaredge
def inlezen_solaredge():
    # Database inlezen
    conn = sqlite3.connect("solaredge.db")
    data = pd.read_sql('SELECT * FROM history', conn)
    
    # Tijd omzetten naar datetime
    data["Time"] = pd.to_datetime(data["tijdstip"])
    data.set_index("Time", inplace=True)
    data.drop("tijdstip", axis=1, inplace=True)
    
    # Tijdzone info toevoegen, SolarEdge is in de "local" time
    data = data.tz_localize("Europe/Amsterdam", ambiguous="NaT", nonexistent="NaT")
        
    return data


# Inlezen gegevens weer
def inlezen_weer():
    # Database inlezen
    conn = sqlite3.connect("knmi.db")
    data = pd.read_sql('SELECT * FROM knmi', conn)
    
    # Date en hour omzetten naar datetime
    data["Time"] = pd.to_datetime(data["date"]) + pd.TimedeltaIndex(data["hour"], unit='h')
    
    data.set_index("Time", inplace=True)
    data.drop(["date", "hour", "id", "station_code"], inplace=True, axis=1)

    # De temperatuur staat in 0.1°C, deze wordt nu door 10 gedeeld om de
    # goede temperatuur te krijgen
    data["T"] = data["T"] / 10
    return data


# Zonnepanelen worden per kwartier gesampled. Met _resample_ het totaal per uur uitrekenen
zonnepanelen = inlezen_solaredge().resample("1H").sum()
    
# Data van het weer inlezen. Samenvoegen met data zonnepanelen
data = inlezen_weer()
data = data.merge(zonnepanelen, left_index=True, right_index=True)

# Nu ook de positie van de zon berekenen op basis van mijn positie
latitude = 51.2
longitude = 6

# pysolar heeft een datetime nodig. Met reset_index van de tijd een kolom maken,
# berekeningen uitvoeren en de tijd kolom weer als index terugzetten
data.reset_index(inplace=True)
# Ik ben er nog niet achter hoe deze functies te _mappen_, dan maar via lambda
data["alt"] = data.apply(
    lambda row: pysolar.solar.get_altitude_fast(
        latitude, longitude, 
        row["Time"].to_pydatetime()), 
    axis=1)
data["azi"] = data.apply(
    lambda row: pysolar.solar.get_azimuth_fast(
        latitude, longitude, 
        row["Time"].to_pydatetime()),
    axis=1)
data.set_index("Time", inplace=True)

# Kolommen een begrijpelijke naam geven
data.rename(columns={"T": "temperatuur", 
             "DR": "duur_neerslag", 
             "N": "bewolking", 
             "alt": "solar_altitude", 
             "azi": "solar_azimuth"},
            inplace=True)

# Dataset opslaan voor te gebruiken in machine learn model
conn = sqlite3.connect("dataset.db")
data_export = data[["temperatuur", "duur_neerslag", "bewolking", "solar_altitude", "solar_azimuth", "energy"]]
data_export.to_sql('history', con=conn, if_exists='replace')
conn.commit()
conn.close()

In [9]:
data_export[data_export["solar_altitude"] > 0]

,temperatuur,duur_neerslag,bewolking,solar_altitude,solar_azimuth,energy
Time,,,,,,
2019-09-16 06:00:00+00:00,12.7,0,8.0,6.117173,94.73043442350287,0.0
2019-09-16 07:00:00+00:00,14.0,0,8.0,15.335837,106.77914882555932,0.0
2019-09-16 08:00:00+00:00,15.4,0,8.0,23.956866,119.82343348027945,0.0
2019-09-16 09:00:00+00:00,16.9,0,8.0,31.441758,134.5231007407539,0.0
2019-09-16 10:00:00+00:00,17.7,0,8.0,37.110053,151.35547060578082,836.0
...,...,...,...,...,...,...
2022-01-31 12:00:00+00:00,5.3,10,8.0,20.979353,182.67539269276645,220.0
2022-01-31 13:00:00+00:00,5.0,10,8.0,19.308456,197.7867962897315,163.0
2022-01-31 14:00:00+00:00,4.9,10,8.0,15.338573,212.16224535752627,63.0
